In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import itertools
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

from sklearn import svm as SVM
from funcs import cv_profits_for_models, cv_preds_and_confusion_matrix, CustomModelWithThreshold
from funcs import profit_scorer, profit_scoring
from customClassifiers import OutlierRemover
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization

from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import chi2

In [2]:
X_train = pd.read_csv('train.csv' ,delimiter="|")
X_test = pd.read_csv('test.csv', delimiter="|")

X_train['scannedLineItemsTotal'] = X_train['scannedLineItemsPerSecond'] * X_train['totalScanTimeInSeconds']
X_train['valuePerLineItem'] = X_train['grandTotal'] * X_train['scannedLineItemsTotal']
X_train['quantityModificationsPerLineItem'] = X_train['quantityModifications'] * X_train['scannedLineItemsTotal']
X_train['lineItemVoids*scansWithoutRegistration'] = X_train['lineItemVoids'] * X_train['scansWithoutRegistration']

In [3]:
X_train1 = X_train[X_train['trustLevel']==1]
y1 = X_train1.pop('fraud')
l=X_train1.pop('trustLevel')

In [4]:
X_train2 = X_train[X_train['trustLevel']==2]
y2 = X_train2.pop('fraud')
l=X_train2.pop('trustLevel')

# Trust Level 1 (Calculate Feature Importance with XGboost)

In [5]:
# normalize data first to prevent 0's in dataset
scaler = StandardScaler()
names_X_train1 = X_train1.columns
X_train1 = scaler.fit_transform(X_train1)

# generate features and rescale
polyFeatures = PolynomialFeatures(3, interaction_only=False)
X_train1_all = polyFeatures.fit_transform(X_train1)
X_train1_all = scaler.fit_transform(X_train1_all)

#remove the first var because it is the constant term
X_train1_all = X_train1_all[:,1:]
features_X_train1_all = polyFeatures.get_feature_names(input_features=names_X_train1)[1:]

/home/niklas/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/niklas/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [6]:
# obtain feature importance by xgboost
xgb = XGBClassifier(num_estimator=100)
xgb.fit(X_train1_all, y1)
imp = xgb.feature_importances_

In [7]:
# order the feature indices by importance
imp = pd.DataFrame(imp)
imp = imp.sort_values(by=0, ascending=False)

In [8]:
# choose starting model
model = CustomModelWithThreshold(LogisticRegression(C=10, solver='lbfgs', max_iter=300), 0.9)

cv = StratifiedKFold(n_splits=10, random_state=42)
last_score = -10000

In [9]:
# add most important feature
X1_temporary = pd.DataFrame(X_train1_all[:,(list(imp.index))[0]])

features_to_use1 = [(list(imp.index))[0]]
# iteratively add features one by one
for featnum in (list(imp.index))[1:]:
    X_check = pd.concat([X1_temporary,pd.Series(X_train1_all[:,featnum])], axis=1)
    score = sum(cross_validate(model,X_check, y1, scoring=profit_scoring, cv=cv)['test_score'])
    # add the feature ultimatively if score improved
    if score > last_score:
        X1_temporary = pd.concat([X1_temporary,pd.Series(X_train1_all[:,featnum])], axis=1)
        features_to_use1.append(featnum)
        last_score = score    
        print(last_score)
    
# for test predictions use features_to_use to select the according features in the test set    

-105
5
15
45
50
90
160
170
180
205
225
245
255
265
275
295
305
315


In [10]:
def evaluateLogReg(C, pred_threshold):
    clf = LogisticRegression(C=C, solver='lbfgs', max_iter=10000)
    clf = CustomModelWithThreshold(clf,threshold=pred_threshold)
    return sum(cross_validate(clf,X1_temporary, y1, scoring=profit_scoring, cv=cv)['test_score'])


params_logreg = {
    'C': (0.1, 5),
    'pred_threshold': (0.5, 0.6)
}

optimization_logreg1 = BayesianOptimization(evaluateLogReg, params_logreg)
optimization_logreg1.maximize(n_iter=100, init_points=100)

|   iter    |  target   |     C     | pred_t... |
-------------------------------------------------
|  1        |  280.0    |  2.401    |  0.5904   |
|  2        |  285.0    |  3.085    |  0.5028   |
|  3        |  280.0    |  1.857    |  0.5667   |
|  4        |  280.0    |  4.918    |  0.5957   |
|  5        |  280.0    |  1.682    |  0.5859   |
|  6        |  285.0    |  3.88     |  0.5321   |
|  7        |  285.0    |  2.64     |  0.5199   |
|  8        |  285.0    |  2.887    |  0.5146   |
|  9        |  285.0    |  1.082    |  0.5955   |
|  10       |  280.0    |  2.374    |  0.5975   |
|  11       |  310.0    |  0.2401   |  0.5328   |
|  12       |  285.0    |  0.8398   |  0.5872   |
|  13       |  295.0    |  1.681    |  0.5056   |
|  14       |  275.0    |  3.057    |  0.5492   |
|  15       |  280.0    |  4.237    |  0.5973   |
|  16       |  300.0    |  1.439    |  0.5481   |
|  17       |  265.0    |  3.961    |  0.5445   |
|  18       |  295.0    |  1.2      |  0.5962   |


KeyboardInterrupt: 

In [11]:
optimization_logreg1.max

{'target': 340.0,
 'params': {'C': 0.3413832354643051, 'pred_threshold': 0.538300775948387}}

In [12]:
# verify

C = optimization_logreg1.max['params']['C']
pred_threshold = optimization_logreg1.max['params']['pred_threshold']

clf = LogisticRegression(C=C, solver='lbfgs', max_iter=10000)
clf = CustomModelWithThreshold(clf,threshold=pred_threshold)
# final score for trust=1
cv = StratifiedKFold(n_splits=10, random_state=42)
sum(cross_validate(clf,X1_temporary, y1, scoring=profit_scoring, cv=cv)['test_score'])

340

In [13]:
# number of features used
len(X1_temporary.columns)

19

In [14]:
features_select_1 = [features_X_train1_all[i] for i in features_to_use1]
features_select_1

['scannedLineItemsTotal',
 'scannedLineItemsPerSecond scannedLineItemsTotal',
 'lineItemVoids scansWithoutRegistration lineItemVoids*scansWithoutRegistration',
 'totalScanTimeInSeconds lineItemVoids*scansWithoutRegistration',
 'valuePerSecond scannedLineItemsTotal^2',
 'scansWithoutRegistration scannedLineItemsTotal',
 'quantityModifications^2 scannedLineItemsTotal',
 'totalScanTimeInSeconds lineItemVoids^2',
 'scansWithoutRegistration lineItemVoidsPerPosition scannedLineItemsTotal',
 'lineItemVoidsPerPosition lineItemVoids*scansWithoutRegistration',
 'totalScanTimeInSeconds scansWithoutRegistration lineItemVoidsPerPosition',
 'lineItemVoids quantityModificationsPerLineItem^2',
 'totalScanTimeInSeconds scannedLineItemsTotal valuePerLineItem',
 'totalScanTimeInSeconds scansWithoutRegistration',
 'totalScanTimeInSeconds lineItemVoids lineItemVoids*scansWithoutRegistration',
 'lineItemVoidsPerPosition scannedLineItemsTotal^2',
 'scannedLineItemsTotal^2',
 'lineItemVoids valuePerSecond qua

# Trust Level 2 (Feature Importance by Logistic Regression)

In [15]:
# normalize data first to prevent 0's in dataset
scaler2 = StandardScaler()
names_X_train2 = X_train2.columns
X_train2 = scaler2.fit_transform(X_train2)

# generate features and rescale
polyFeatures2 = PolynomialFeatures(3, interaction_only=False)
X_train2_all = polyFeatures2.fit_transform(X_train2)
X_train2_all = scaler.fit_transform(X_train2_all)

#remove the first var because it is the constant term
X_train2_all = X_train2_all[:,1:]
features_X_train2_all = polyFeatures2.get_feature_names(input_features=names_X_train2)[1:]


/home/niklas/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/niklas/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [16]:
# Feature Importance with logistic Regression
lr = LogisticRegression(C=20, solver='lbfgs')
lr.fit(X_train2_all, y2)
imp = lr.coef_[0]

In [17]:
# order the feature indices by importance
imp = pd.DataFrame(imp)
imp = imp.sort_values(by=0, ascending=False)

In [18]:
# choose starting model
model = CustomModelWithThreshold(LogisticRegression(C=10, solver='lbfgs', max_iter=300), 0.9)

cv = StratifiedKFold(n_splits=10, random_state=42)
last_score = -10000

In [19]:
# add most important feature
X2_temporary = pd.DataFrame(X_train2_all[:,(list(imp.index))[0]])

features_to_use2 = [(list(imp.index))[0]]
# iteratively add features one by one
for featnum in (list(imp.index))[1:]:
    X_check = pd.concat([X2_temporary,pd.Series(X_train2_all[:,featnum])], axis=1)
    score = sum(cross_validate(model,X_check, y2, scoring=profit_scoring, cv=cv)['test_score'])
    # add the feature ultimatively if score improved
    if score > last_score:
        X2_temporary = pd.concat([X2_temporary,pd.Series(X_train2_all[:,featnum])], axis=1)
        features_to_use2.append(featnum)
        last_score = score    
        print(last_score)
    
# for test predictions use features_to_use to select the according features in the test set    

-75
-55
-45
-25
15
35


In [20]:
# dataset in function is X2
def evaluateLogReg(C, pred_threshold):
    clf = LogisticRegression(C=C, solver='lbfgs', max_iter=10000)
    clf = CustomModelWithThreshold(clf,threshold=pred_threshold)
    return sum(cross_validate(clf,X2_temporary, y2, scoring=profit_scoring, cv=cv)['test_score'])


params_logreg = {
    'C': (0.001, 50),
    'pred_threshold': (0.5, 1)
}

optimization_logreg = BayesianOptimization(evaluateLogReg, params_logreg)
optimization_logreg.maximize(n_iter=100, init_points=100)

|   iter    |  target   |     C     | pred_t... |
-------------------------------------------------
|  1        |  10.0     |  23.03    |  0.8011   |
|  2        |  10.0     |  15.12    |  0.5295   |
|  3        |  10.0     |  13.01    |  0.5812   |
|  4        |  35.0     |  47.47    |  0.9132   |
|  5        |  10.0     |  32.4     |  0.822    |
|  6        |  45.0     |  2.664    |  0.5811   |
|  7        |  35.0     |  2.251    |  0.7448   |
|  8        |  10.0     |  47.06    |  0.6282   |
|  9        |  35.0     |  23.45    |  0.8892   |
|  10       |  10.0     |  42.75    |  0.678    |
|  11       |  5.0      |  4.683    |  0.9269   |
|  12       |  10.0     |  44.95    |  0.641    |
|  13       |  10.0     |  13.57    |  0.7491   |
|  14       |  35.0     |  12.87    |  0.8224   |
|  15       |  10.0     |  25.48    |  0.8071   |
|  16       |  10.0     |  22.85    |  0.5979   |
|  17       |  10.0     |  31.14    |  0.6291   |
|  18       |  25.0     |  14.91    |  0.9434   |


KeyboardInterrupt: 

In [21]:
optimization_logreg.max

{'target': 45.0,
 'params': {'C': 2.6642719484799917, 'pred_threshold': 0.5810930799041378}}

In [22]:
# verify
C = optimization_logreg.max['params']['C']
pred_threshold = optimization_logreg.max['params']['pred_threshold']

clf = LogisticRegression(C=C, solver='lbfgs', max_iter=10000)
clf = CustomModelWithThreshold(clf,threshold=pred_threshold)
# final score for trust=2
cv = StratifiedKFold(n_splits=10, random_state=42)
sum(cross_validate(clf,X2_temporary, y2, scoring=profit_scoring, cv=cv)['test_score'])

45

In [23]:
# number of features used
len(X2_temporary.columns)

7

In [24]:
features_select_2 = [features_X_train2_all[i] for i in features_to_use2]
features_select_2

['scannedLineItemsTotal^2',
 'scannedLineItemsTotal^3',
 'totalScanTimeInSeconds scannedLineItemsTotal^2',
 'totalScanTimeInSeconds lineItemVoids scannedLineItemsTotal',
 'totalScanTimeInSeconds scannedLineItemsTotal valuePerLineItem',
 'totalScanTimeInSeconds scansWithoutRegistration valuePerLineItem',
 'scannedLineItemsTotal^2 lineItemVoids*scansWithoutRegistration']